In [91]:
#Based on the implementation found:
#https://dev.to/mxl/dijkstras-algorithm-in-python-algorithms-for-beginners-dkc
from collections import deque, namedtuple

# we'll use infinity as a default distance to nodes.
inf = float('inf')
Edge = namedtuple('Edge', 'start, end, cost, capacity')

import sched, time
s = sched.scheduler(time.time, time.sleep)

def print_time(a='default'):
    print("From print_time", time.time(), a)

#defining a global threshold time and capacity for each of the channels
#will most probably vary in the future
timeThreshold = 5
originalCapacity = 1

def make_edge(start, end, cost=1, capacity=1):
    return Edge(start, end, cost, capacity)

def make_backward_edge(start, end, cost=1, capacity=1):
    return Edge(end, start, cost, capacity)

class Graph:
    def __init__(self, edges):
        # let's check that the data is right
        wrong_edges = [i for i in edges if len(i) not in [2, 4]]
        if wrong_edges:
            raise ValueError('Wrong edges data: {}'.format(wrong_edges))

        self.edges = [make_edge(*edge) for edge in edges] + [make_backward_edge(*edge) for edge in edges]

    # @property:
    # https://www.programiz.com/python-programming/property 
    @property
    def vertices(self):
        return set(
            sum(
                ([edge.start, edge.end] for edge in self.edges), []
            )
        )

    def get_node_pairs(self, n1, n2, both_ends=True):
        if both_ends:
            node_pairs = [[n1, n2], [n2, n1]]
        else:
            node_pairs = [[n1, n2]]
        return node_pairs

    
    def add_edge(self, n1, n2, cost=1, capacity=1, both_ends=True):
        node_pairs = self.get_node_pairs(n1, n2, both_ends)
        for edge in self.edges:
            if [edge.start, edge.end] in node_pairs:
                return ValueError('Edge {} {} already exists'.format(n1, n2))

        self.edges.append(Edge(start=n1, end=n2, cost=cost, capacity=capacity))
        if both_ends:
            self.edges.append(Edge(start=n2, end=n1, cost=cost, capacity=capacity))
    
    def remove_edge(self, n1, n2, both_ends=True):
        
        node_pairs = self.get_node_pairs(n1, n2, both_ends)
        edges = self.edges[:]
        for edge in edges:
            if [edge.start, edge.end] in node_pairs:
                
                #We need to wait if currently there is no available link through the channel
                if edge.capacity == 0:
                    s.run()
                    
                #we have used a channel along this edge, remove one edge from here
                self.edges.remove(edge)
                self.edges.append(make_edge(edge.start, edge.end,edge.cost,edge.capacity-1))
                
                #start a timer with the threshold time and then reset the edge with originalCapacity
                s.enter(timeThreshold, 1, add_edge(self, edge.start, edge.end, cost=1, capacity=edge.capacity+10000, both_ends=True))
        return edges

    @property
    def neighbours(self):
        neighbours = {vertex: set() for vertex in self.vertices}
        for edge in self.edges:
            neighbours[edge.start].add((edge.end, edge.cost))

        return neighbours

    def dijkstra(self, source, dest):
        
        s.enter(5, 1, print_time)
        s.run()
        
        #Checking if the given source is existing or not
        assert source in self.vertices, 'Such source node doesn\'t exist'

        #Running the Initialize-Single-Source procedure
        distances = {vertex: inf for vertex in self.vertices}
        previous_vertices = {
            vertex: None for vertex in self.vertices
        }
        distances[source] = 0
        vertices = self.vertices.copy()

        #picking each vertex out of the min-priority deque
        while vertices:
            current_vertex = min(
                vertices, key=lambda vertex: distances[vertex])
            vertices.remove(current_vertex)
            
            #Relaxation procedure
            if distances[current_vertex] == inf:
                break
            for neighbour, cost in self.neighbours[current_vertex]:
                alternative_route = distances[current_vertex] + cost
                if alternative_route < distances[neighbour]:
                    distances[neighbour] = alternative_route
                    previous_vertices[neighbour] = current_vertex

        path, current_vertex = deque(), dest
        while previous_vertices[current_vertex] is not None:
            path.appendleft(current_vertex)
            current_vertex = previous_vertices[current_vertex]
        if path:
            path.appendleft(current_vertex)
        return path

In [92]:
#print(graph.dijkstra("a", "f"))
#print(graph.edges)
graph.remove_edge("a","c")
#print(graph.edges)
#update state according to result

NameError: name 'add_edge' is not defined

In [89]:
#Defining the graph
#Vertex names: a-h
graph = Graph([
       
    #Initialize circular graph
    ("a", "b"),  ("b", "c"),   ("c", "d"), ("d", "e"), ("e", "f"), ("f", "g"), ("g", "h"), ("h", "a"),


    #Add "diagonal" edges
    ("a", "c"), ("d", "e"), ("e", "g"),  ("g", "a")])

graphOriginalEdges = graph.edges.copy()
print(graphOriginalEdges)

[Edge(start='a', end='b', cost=1, capacity=1), Edge(start='b', end='c', cost=1, capacity=1), Edge(start='c', end='d', cost=1, capacity=1), Edge(start='d', end='e', cost=1, capacity=1), Edge(start='e', end='f', cost=1, capacity=1), Edge(start='f', end='g', cost=1, capacity=1), Edge(start='g', end='h', cost=1, capacity=1), Edge(start='h', end='a', cost=1, capacity=1), Edge(start='a', end='c', cost=1, capacity=1), Edge(start='d', end='e', cost=1, capacity=1), Edge(start='e', end='g', cost=1, capacity=1), Edge(start='g', end='a', cost=1, capacity=1), Edge(start='b', end='a', cost=1, capacity=1), Edge(start='c', end='b', cost=1, capacity=1), Edge(start='d', end='c', cost=1, capacity=1), Edge(start='e', end='d', cost=1, capacity=1), Edge(start='f', end='e', cost=1, capacity=1), Edge(start='g', end='f', cost=1, capacity=1), Edge(start='h', end='g', cost=1, capacity=1), Edge(start='a', end='h', cost=1, capacity=1), Edge(start='c', end='a', cost=1, capacity=1), Edge(start='e', end='d', cost=1, 

In [ ]:
#TODO:
#1. create topology
#2. create timestamp objects
# a) add the timestamps and 

#3. calculate EDT
#4. remove the edges used

#+1: always have the updated info
#+2: create Jupyter notebook

#Are the packets sent simultanously, or one after the other?



In [46]:
import sched, time
s = sched.scheduler(time.time, time.sleep)
def print_time(a='default'):
    print("From print_time", time.time(), a)

def print_some_times():
    print(time.time())
    s.enter(10, 1, print_time)
    s.enter(5, 2, print_time, argument=('positional',))
    s.enter(5, 1, print_time, kwargs={'a': 'keyword'})
    s.run()
    print(time.time())
    
print_some_times()

1545019551.3211548
From print_time 1545019556.3238869 positional
From print_time 1545019556.3239796 keyword
From print_time 1545019561.3238668 default
1545019561.323943
